In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import model_selection


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

In [ ]:
!unzip /content/plantvillage-dataset.zip

In [ ]:
import imageio as iio
imag=iio.imread('/content/plantvillage dataset/color/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG')
imag.shape

In [ ]:
pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('/content/plantvillage dataset/color', output="output", seed=1337, ratio=(.8, .2), group_prefix=None)

In [ ]:
train=tf.keras.preprocessing.image_dataset_from_directory('/content/output/train',label_mode='categorical',
                                                          image_size=[256,256],batch_size=32,shuffle=1000,seed=42)

In [ ]:
val=tf.keras.preprocessing.image_dataset_from_directory('/content/output/val',
                                                        image_size=[256,256],
                                                        shuffle=1000,batch_size=32,label_mode='categorical')

In [ ]:
for img,label in train.take(1):
  print(img[0])


In [ ]:
def preprocess(img,label):
  img=img/255
  print(img,label)
  dataset=dataset.map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
def img_augmentation(img,label):
  img=tf.image.random_flip_left_right(img)
  img=tf.image.random_flip_up_down(img)
  img=tf.image.random_brightness(img,max_delta=.2)
  img=tf.clip_by_value(img,0,1)
  return img,label

In [ ]:
def prepare_dataset(dataset,training):
  # dataset=dataset.map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset.prefetch(tf.data.AUTOTUNE)
  if training:
    dataset=dataset.map(img_augmentation)
    return dataset
train_dataset=prepare_dataset(train,training=True)
test_dataset=prepare_dataset(val,training=False)

In [ ]:
len(train_dataset)

In [ ]:
for img,label in train_dataset.take(1):
  print (label[0])
  print(img[0])

#Modeling

In [ ]:
model_cnn=tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(filters=32,kernel_size=3,
                               activation='relu',padding='same',kernel_initializer='he_normal',input_shape=[256,256,3]
            
        ),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same', kernel_initializer='he_normal',activation='relu'),
     tf.keras.layers.MaxPool2D(pool_size=(2,2)),
     tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu',padding='same'),
     tf.keras.layers.MaxPool2D(pool_size=(2,2)),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(256,activation='relu'),
     tf.keras.layers.Dense(38,activation='softmax')


     
    ]
)

In [ ]:
model_cnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.001,beta_1=.9,beta_2=.999),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.metrics.CategoricalAccuracy()]
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True)
LR=tf.keras.callbacks.ReduceLROnPlateau(mointor="val_loss",
                                        factor=.1,
                                        patience=7)


model_cnn.fit(train_dataset,
                    validation_data = test_dataset,epochs=100,callbacks=[early_stopping,LR])